# 문장간 유사도 검사

In [ ]:
!pip install -U scikit-learn
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import konlpy
from konlpy.tag import Okt
t = Okt()

vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

def sentence_token(contents):
  contents_tokens = [t.morphs(row) for row in contents]

  contents_for_vectorize = []
  for content in contents_tokens:
    sentence = ''
    for word in content:
      sentence = sentence + ' ' + word
    contents_for_vectorize.append(sentence)

  #tf-idf 벡터화
  X = vectorizer.fit_transform(contents_for_vectorize)
  num_samples, num_features = X.shape
  print(num_samples, num_features)
  return X


In [ ]:
def new_token(new_post):
  new_post_tokens = [t.morphs(row) for row in new_post]
  new_post_for_vectorize = []
  for content in new_post_tokens:
    sentence = ''
    for word in content:
      sentence = sentence + ' ' + word
    new_post_for_vectorize.append(sentence)
  new_post_vec = vectorizer.transform(new_post_for_vectorize)
  print(new_post_for_vectorize)
  return new_post_vec

In [ ]:
import scipy as sp
def dist_raw(v1,v2):
  delta = v1 - v2
  return sp.linalg.norm(delta.toarray())

In [ ]:
def check_distance(X, new_post_vec, contents):
  best_doc = None
  best_dist = 65535
  best_i = None
  result = []
  for i in range(len(contents)):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    #print(d)
    print('== Post %i with dist = %.2f : %s' %(i,d, contents[i]))
    if d < best_dist:
      best_dist = d
      best_i = i
      result = []
    if d == best_dist:
      result.append(contents[i])
  return best_i, best_dist, result

In [ ]:
contents = [
    "작년 대비 소비자물가가 33% 증가했다.",
    "작년 대비 소비자물가가 33%증가했다.",
    "작년 대비 소비자물가가 15% 증가했다.",
    "내 나이는 올해 33살이다.",
    "문방구가서 펜 33개만 사와줄래?",
    "소비자물가가 크게 오를 전망이다.",
    "작년에 비해 소비자물가가 증가했다.",
    "작년에 비해 소비자물가가 33% 증가했다."
]
new_post = ["작년 대비 소비자물가가 33% 증가했다."]

X = sentence_token(contents)
new_post_vec = new_token(new_post)
best_i, best_dist, result = check_distance(X, new_post_vec, contents)
print("Best post is %i, dist=%.2f" %(best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

8 19
[' 작년 대비 소비자 물가 가 33% 증가 했다 .']
== Post 0 with dist = 0.00 : 작년 대비 소비자물가가 33% 증가했다.
== Post 1 with dist = 0.00 : 작년 대비 소비자물가가 33%증가했다.
== Post 2 with dist = 0.70 : 작년 대비 소비자물가가 15% 증가했다.
== Post 3 with dist = 1.33 : 내 나이는 올해 33살이다.
== Post 4 with dist = 1.33 : 문방구가서 펜 33개만 사와줄래?
== Post 5 with dist = 1.30 : 소비자물가가 크게 오를 전망이다.
== Post 6 with dist = 0.83 : 작년에 비해 소비자물가가 증가했다.
== Post 7 with dist = 0.72 : 작년에 비해 소비자물가가 33% 증가했다.
Best post is 0, dist=0.00
--> ['작년 대비 소비자물가가 33% 증가했다.']
----> 작년 대비 소비자물가가 33% 증가했다.


# 네이버 맞춤법 검사기

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-46wpsglf
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-46wpsglf
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4810 sha256=4115644bfbc92ebb6bfd87290895186b1e16d9b9a96692080edfb74db4694ad3
  Stored in directory: /tmp/pip-ephem-wheel-cache-fu_jefsl/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker
def spell_checking(result):
  for i in range(len(result)):
    test = spell_checker.check(result[i]) #각 사용자 답안별로 확인
    test = test.as_dict()
    for word, error in test['words'].items():
      if error == 0:
          print(f"{word}: 통과")
      elif error == 1:
          print(f"{word}: 맞춤법에 문제가 있는 단어 또는 구절")
      elif error == 2:
          print(f"{word}: 띄어쓰기에 문제가 있는 단어 또는 구절")
      elif error == 3:
          print(f"{word}: 표준어가 의심되는 단어 또는 구절")
      elif error == 4:
          print(f"{word}: 통계적 교정에 따른 단어 또는 구절")
    print('=====')
spell_checking(result)

작년: 통과
대비: 통과
소비자물가가: 통과
33%: 통과
증가했다.: 통과
=====
작년: 통과
대비: 통과
소비자물가가: 통과
33%: 띄어쓰기에 문제가 있는 단어 또는 구절
증가했다.: 띄어쓰기에 문제가 있는 단어 또는 구절
=====


In [ ]:
contents = ['''인주시의 자전거 이용자 변화를 살펴보면, 자전거 이용자 수는 2007년 4만 명에서 2012년에는 9만 명, 2017년에는 21만 명으로, 지난 10년간 약 5배 증가하였다.
  특히 2012년분터 2017년까지 자전거 이용하는 사람수가 급증한 것으로 나타났다. 이와 같이 자전거 이용자 수가 증가한 이유는 자전거 도로가 개발돼고 자전거 빌리는 곳이 홝대됐기 때문인 것으로 보인다.
  자전거 이용 목적을 보면, 10년간 운동 과 산책은 4배, 출퇴근은 14배, 기타는 3배 늘어난 것으로 나타났으며, 출퇴근시 이용이 가장 높은 증가율을 보였다.''']

new_post = ['''인주시의 자전거 이용자 변화를 살펴보면, 자전거 이용자 수는 2007년 4만 명에서 2012년에는 9만명, 2017년에는 21만명으로, 지난 10년간 약 5배 증가하였다.
  특히 2012년분터 2017년까지 자전거 이용자 수가 급증한 것으로 나타났다. 이와 같이 자전거 이용자 수가 증가한 이유는 자전거 도로가 개발되고 자전거 빌리는 곳이 확대되었기 때문인 것으로 보인다.
  자전거 이용 목적을 보면, 10년간 운동 및 산책은 4배, 출퇴근은 14배, 기타는 3배 늘어난 것으로 나타났으며, 출퇴근 시 이용이 가장 높은 증가율을 보였다.''']

X = sentence_token(contents)
new_post_vec = new_token(new_post)
best_i, best_dist, result = check_distance(X, new_post_vec, contents)
print("Best post is %i, dist=%.2f" %(best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

1 48
[' 인주시의 자전거 이용자 변화 를 살펴보면 , 자전거 이용자 수 는 2007년 4만 명 에서 2012년 에는 9만 명 , 2017년 에는 21만 명으로 , 지난 10년 간 약 5 배 증가 하였다 . \n   특히 2012년 분터 2017년 까지 자전거 이용자 수가 급증 한 것 으로 나타났다 . 이 와 같이 자전거 이용자 수가 증가 한 이유 는 자전거 도로 가 개발 되고 자전거 빌리는 곳 이 확대 되었기 때문 인 것 으로 보인다 . \n   자전거 이용 목적 을 보면 , 10년 간 운동 및 산책 은 4 배 , 출퇴근 은 14 배 , 기타 는 3 배 늘어난 것 으로 나타났으며 , 출퇴근 시 이용 이 가장 높은 증가 율 을 보였다 .']
== Post 0 with dist = 0.22 : 인주시의 자전거 이용자 변화를 살펴보면, 자전거 이용자 수는 2007년 4만 명에서 2012년에는 9만 명, 2017년에는 21만 명으로, 지난 10년간 약 5배 증가하였다.
  특히 2012년분터 2017년까지 자전거 이용하는 사람수가 급증한 것으로 나타났다. 이와 같이 자전거 이용자 수가 증가한 이유는 자전거 도로가 개발돼고 자전거 빌리는 곳이 홝대됐기 때문인 것으로 보인다.
  자전거 이용 목적을 보면, 10년간 운동 과 산책은 4배, 출퇴근은 14배, 기타는 3배 늘어난 것으로 나타났으며, 출퇴근시 이용이 가장 높은 증가율을 보였다.
Best post is 0, dist=0.22
--> ['인주시의 자전거 이용자 변화를 살펴보면, 자전거 이용자 수는 2007년 4만 명에서 2012년에는 9만명, 2017년에는 21만명으로, 지난 10년간 약 5배 증가하였다.\n  특히 2012년분터 2017년까지 자전거 이용자 수가 급증한 것으로 나타났다. 이와 같이 자전거 이용자 수가 증가한 이유는 자전거 도로가 개발되고 자전거 빌리는 곳이 확대되었기 때문인 것으로 보인다.\n  자전거 이용 목적을 보면, 10년간 운동 및 산책은 4배, 출퇴근은 14배, 기타는 3배 늘어난 

In [ ]:
print(result[0])

인주시의 자전거 이용자 변화를 살펴보면, 자전거 이용자 수는 2007년 4만 명에서 2012년에는 9만 명, 2017년에는 21만 명으로, 지난 10년간 약 5배 증가하였다.
  특히 2012년분터 2017년까지 자전거 이용하는 사람수가 급증한 것으로 나타났다. 이와 같이 자전거 이용자 수가 증가한 이유는 자전거 도로가 개발돼고 자전거 빌리는 곳이 홝대됐기 때문인 것으로 보인다.
  자전거 이용 목적을 보면, 10년간 운동 과 산책은 4배, 출퇴근은 14배, 기타는 3배 늘어난 것으로 나타났으며, 출퇴근시 이용이 가장 높은 증가율을 보였다.


# 부산대 맞춤법 검사기

In [ ]:
def split_sentence(text):
  sentences = text.split('.')
  result = []
  current_sentence = sentences[0]
  for sentence in sentences[1:]:
    if len(current_sentence) + len(sentence) < 500:
      current_sentence += '. ' + sentence
    else:
      result.append(current_sentence)
      current_sentence = sentence
  result.append(current_sentence)
  return result

In [ ]:
result_split = split_sentence(result[0])

In [ ]:
print(result_split[0])

인주시의 자전거 이용자 변화를 살펴보면, 자전거 이용자 수는 2007년 4만 명에서 2012년에는 9만 명, 2017년에는 21만 명으로, 지난 10년간 약 5배 증가하였다. 
  특히 2012년분터 2017년까지 자전거 이용하는 사람수가 급증한 것으로 나타났다.  이와 같이 자전거 이용자 수가 증가한 이유는 자전거 도로가 개발돼고 자전거 빌리는 곳이 홝대됐기 때문인 것으로 보인다. 
  자전거 이용 목적을 보면, 10년간 운동 과 산책은 4배, 출퇴근은 14배, 기타는 3배 늘어난 것으로 나타났으며, 출퇴근시 이용이 가장 높은 증가율을 보였다. 


In [ ]:
import requests
import json

for i in range(len(result_split)):
  results = result_split[i].replace('\n', '\r\n')

  response = requests.post('http://164.125.7.61/speller/results', data={'text1': results})

  data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0]

  data = json.loads(data)


  for err in data['errInfo']:
      print(f"입력 내용 : {err['orgStr']}")
      print(f"대치어 : {err['candWord']}")
      print(f"도움말 : {err['help']}") #여기 apos; 나오고 <br/>이런거 나오는데 혹시 수정해야되면 메시지 작성 바람.
      print("\n")

입력 내용 : 인주시의
대치어 : 인주 시의|인주기의
도움말 : 철자 검사를 해 보니 이 어절은 분석할 수 없으므로 틀린 말로 판단하였습니다.<br/><br/>후보 어절은 이 철자검사/교정기에서 띄어쓰기, 붙여 쓰기, 음절대치와 같은 교정방법에 따라 수정한 결과입니다.<br/><br/>후보 어절 중 선택하시거나 오류 어절을 수정하여 주십시오.<br/><br/>* 단, 사전에 없는 단어이거나 사용자가 올바르다고 판단한 어절에 대해서는 통과하세요!!


입력 내용 : 2012년분터
대치어 : 2012년부터
도움말 : 조사 오류입니다.


입력 내용 : 사람수가
대치어 : 사람 수가
도움말 : &apos;수&apos;는 &apos;숫자&apos;의 준말로 앞 단어가 &apos;수&apos;를 꾸미는 역할을 합니다. 두 단어 사이에 관형격 조사 &apos;의&apos;를 넣을 수 있으므로 띄어 써야 바릅니다.


입력 내용 : 개발돼고
대치어 : 개발됐고|개발되고|개발돼도
도움말 : 어미의 사용이 잘못되었습니다. 문서 작성시 필요에 의해 잘못된 어미를 제시하는 상황이 아니라면 검사기의 대치어로 바꾸도록 합니다.


입력 내용 : 홝대됐기
대치어 : 확대됐기
도움말 : 입력 오류입니다.


입력 내용 : 때문인 것으로
대치어 : 때문으로
도움말 : &apos;-인 것&apos;, &apos;-할 것&apos; 등은 확실힌 명제를 뜻없이 격식화한 표현입니다. 이런 표현은 자신의 주장을 남의 일처럼 말하는 격이므로 좋은 표현이 아닙니다. 그러므로 문맥에 적절한 표현으로 바꾸어 쓰십시오.


입력 내용 : 운동 과
대치어 : 운동과
도움말 : 명사 다음에 붙는 공동격 조사를 쓸 때, 받침이 없을 때는 &apos;와&apos;를 쓰고, 있을 때는 &apos;과&apos;를 써야 바릅니다. 또한 조사는 앞에 오는 명사와 붙여 써야 바릅니다.


입력 내용 : 출퇴근시
대치어 : 출퇴근 시|출퇴근 때|출퇴근할 때
도움말 : 동작 명사 뒤에 오는 &apos;중&apos;과 &apo